# Procurando Bares com GeoDjango

Planejando suas atividades pós-quarentena!

https://docs.djangoproject.com/en/3.0/ref/contrib/gis/

In [4]:
import os
import sys
import django

# Adiciona diretório no projeto no PYTHONPATH
sys.path.append(os.path.join(os.getcwd(), 'geo'))

# Definine settings Django a ser utilizado
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'geo.settings')

# Configura Django para rodar no Notebook
django.setup()

## Habilitar uso do GeoDjango

Incluir `django.contrib.gis` no settings `INSTALLED_APPS` para habilitar *management commands*, uso de templates, etc.

```python
INSTALLED_APPS = [
    'django.contrib.gis',
]
```

Modificar `ENGINE` de `DATABASES` para `django.contrib.gis.db.backends.postgis`

```python
DATABASES = {
    'default': {
        'ENGINE': 'django.contrib.gis.db.backends.postgis',
        'NAME': 'geo',
        'USER': 'postgres',
        'PASSWORD': 'postgres'
    }
}
```

## Baixar dados do OpenStreetMap

Mapa colaborativo https://www.openstreetmap.org com licença de uso aberta.

Vamos baixar dados utilizando o Overpass Turbo https://overpass-turbo.eu/, neste caso vamos escolher somente a tag `amenity` que tenha valor `bar`.

![Overpass Turbo](./images/overpass.png)

Podemos baixar os dados no formato GeoJSON e importá-los para nosso banco de dados. Em nosso caso, gravamos um arquivo `bar.geojson`.

Veja um trecho do arquivo baixado:

```json
{
  "type": "FeatureCollection",
  "generator": "overpass-ide",
  "copyright": "The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.",
  "timestamp": "2020-06-15T19:05:02Z",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "@id": "node/319258499",
        "amenity": "bar",
        "name": "Bar do Salomão"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -43.9198583,
          -19.9388896
        ]
      },
      "id": "node/319258499"
    },
    {
      "type": "Feature",
      "properties": {
        "@id": "node/745741462",
        "addr:street": "Avenida Engenheiro Caetano Alvares",
        "amenity": "bar",
        "name": "Bar do Luiz"
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          -46.6386645,
          -23.4860942
        ]
      },
      "id": "node/745741462"
    }
}
```

## Modelo POI (Point of Interest)

Criamos um modelo `POI` com alguns campos retornados no GeoJSON. Pode-se usar o comando [ogrinspect](https://docs.djangoproject.com/en/3.0/ref/contrib/gis/commands/#ogrinspect) `manage.py ogrinspect bar.geojson POI` para gerar um modelo baseado no arquivo GeoJSON baixado, porém serve somente para um rascunho inicial, já que ele não tem como identificar os tamanhos dos campos caracteres e gera campos com nomes da maneira que estão no arquivo, com `:` no nome, que é código Python inválido.

Nosso modelo final ficou assim:

```python
from django.contrib.gis.db import models


class POI(models.Model):
    id = models.CharField(max_length=50, primary_key=True)
    amenity = models.CharField(max_length=50)
    name = models.CharField(max_length=200, blank=True, default='')
    addr_street = models.CharField(max_length=300, default='')
    addr_city = models.CharField(max_length=100, default='')
    addr_country = models.CharField(max_length=100, default='')
    addr_housenumber = models.CharField(max_length=30,  default='')
    wheelchair = models.CharField(max_length=10, default='')
    addr_postcode = models.CharField(max_length=10, default='')
    phone = models.CharField(max_length=50, default='')
    source = models.CharField(max_length=100, default='')
    brewery = models.CharField(max_length=100, default='')
    smoking = models.CharField(max_length=50, default='')
    food = models.CharField(max_length=50, default='')
    internet_access = models.CharField(max_length=50, default='')
    website = models.CharField(max_length=300, default='')
    outdoor_seating = models.CharField(max_length=50, default='')
    delivery = models.CharField(max_length=50, default='')
    description = models.CharField(max_length=300, default='')
    addr_state = models.CharField(max_length=100, default='')
    opening_hours_covid19 = models.CharField(max_length=50, default='')
    geom = models.PointField(srid=4326)
```

## Importando dados para o Banco

Importamos os dados utilizando um `management command` [loadbars](/edit/work/geo/poi/management/commands/loadbars.py)

Chamado desta maneira:

```
/manage.py loadbars ../bar.geojson
```

## Navegando nos dados

In [5]:
from poi.models import POI

# Quantos registros vieram
POI.objects.count()

973

In [6]:
from django.contrib.gis.geos import Point
from django.contrib.gis.db.models.functions import Distance

ponto = Point(-45.851157, -23.192602, srid=4236)

# Bares mais próximos
top_10 = POI.objects.annotate(distancia=Distance('geom', ponto)).order_by('distancia')[:10]
for bar in top_10:
    print(f'{bar}: {bar.distancia}, {bar.opening_hours_covid19}')

Bar Recanto Nordestino: 1804.26290238 m, None
BeerGarden: 2951.70045242 m, None
Empório Hopfields: 3043.44738127 m, None
Guten Bier: 3593.00705239 m, None
Disk Bebidas: 3719.66833005 m, None
Nibs Juice Bar: 3789.84072914 m, None
Capital da Vila: 3922.47238593 m, None
The Drunk Monk: 3929.05305045 m, None
Mestre Cervejeiro: 3975.50496183 m, None
Baltazar: 4403.78509042 m, None


In [8]:
cov = POI.objects.filter(opening_hours_covid19__isnull=False)

In [9]:
for bar in cov:
    print(f'{bar}')
    

--
Beco do Espeto
Vinicula Torre Alta
